In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
from numpy.random import randn
from matplotlib import pyplot
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as k

import csv
import cv2
import os

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201 ,DenseNet121
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split

from scipy import ndimage

In [2]:
model = load_model("../input/croped-faces/generator_model_280k.h5")

In [3]:
def generate_laten (latent_dim , n_sample):
    x_input = randn(latent_dim * n_sample)
    x_input = x_input.reshape(n_sample , latent_dim)
    return x_input

In [4]:
def save_plot(examples , n):
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.axis('off')
        plt.imshow(examples[i, : , : , 0 ] )
    plt.show()

In [5]:
latent_point = generate_laten(100,2600)
x = model.predict(latent_point)
save_plot(x, 4)

In [6]:
latent_point = generate_laten(100,900)
y = model.predict(latent_point)
save_plot(y, 4)

In [7]:
x.shape

In [8]:
type(x)

In [9]:
path_fer = "../input/fer2013/train/"
t_path = "../input/fer2013/test/"


In [10]:
imagetype= os.listdir("../input/fer2013/train")
imagetype

In [11]:
images = []    #images for train list will store all images
labels= []
t_images = []    #images for test list will store all images
t_labels= []

In [12]:
for i in imagetype:
    print(i)
    flower_path=path_fer+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".jpg") ]
    counter=0
    for data in data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img = cv2.resize(img , (64,64))
        images.append(img)
        labels.append(i)
print("total train images:",len(images))
print("total train images:",len(labels))

In [13]:
for i in imagetype:
    print(i)
    flower_path=t_path+str(i)
    test_data_list=[j for j in os.listdir(flower_path) if j.endswith(".jpg") ]
    counter=0
    for data in test_data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img = cv2.resize(img , (64,64))
        t_images.append(img)
        t_labels.append(i)
print("total test images:",len(t_images))
print("total test images:",len(t_labels))

In [14]:
for i in x:
    images.append(img)
    labels.append("disgust")

In [15]:
for i in y:
    t_images.append(img)
    t_labels.append("disgust")

In [16]:
images=np.array(images)
t_images=np.array(t_images)
images = images/ 255
t_images = t_images/ 255
print(images.shape)
print(t_images.shape)

In [17]:
encode= LabelEncoder()
labels=encode.fit_transform(labels)
encode2= LabelEncoder()
t_labels = encode2.fit_transform(t_labels)
t_labels[1:10]

In [18]:
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
print("train labels is ready!")

In [19]:
t_labels=t_labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
t_labels=onehotencoder.fit_transform(t_labels)
t_labels=t_labels.toarray()
print("test labels is ready!")
t_labels[:10]

In [20]:
images,labels = shuffle(images, labels, random_state=1)
t_images , t_labels =  shuffle(t_images, t_labels, random_state=1)
plt.imshow(images[4])
plt.xlabel(labels[4])

In [21]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=True):
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))

In [22]:
class_weights = generate_class_weights(labels)

In [23]:
class_weights

In [24]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=None,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0,
#     zoom_range=0.1,
    horizontal_flip=True
)
train_data_gen.fit(images)

In [25]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),
  layers.Dropout(0.5), 


#CNN layer 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.5), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [26]:
opt = keras.optimizers.Adamax(learning_rate=0.0001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
# history = model.fit(images , labels , batch_size=32 , epochs=32 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [27]:
history = model.fit(images , labels , batch_size=32 , epochs=32 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [28]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')
# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [29]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),
  layers.Dropout(0.65), 


#CNN layer 

#   layers.Conv2D(128,(3,3),padding = 'same'), 
#   layers.BatchNormalization(),
#   layers.LeakyReLU(alpha=0.1),
#   layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#   layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.5), 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.5), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [30]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
history = model.fit(images , labels , batch_size=128 , epochs=32 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [31]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')
# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [24]:
model = keras.Sequential([
# Data Augmentation 
  

  VGG16(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [25]:
opt = keras.optimizers.SGD(learning_rate=0.001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [26]:
history = model.fit(images , labels , batch_size=32 , epochs=32 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [27]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [29]:
history = model.fit(images , labels , batch_size=32 , epochs=50 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [30]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [43]:
model = keras.Sequential([
# Data Augmentation 
  

  keras.applications.ResNet50(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

#   layers.Conv2D(128,(3,3),padding = 'same'), 
#   layers.BatchNormalization(),
#   layers.LeakyReLU(alpha=0.1),
#   layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#   layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(16),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [45]:
opt = keras.optimizers.SGD(learning_rate=0.001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
history = model.fit(images , labels , batch_size=64 , epochs=50 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [46]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [52]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D

model = keras.Sequential()

# Conv Block 1
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Conv Block 2
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Conv Block 3
model.add(Conv2D(256,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Conv Block 3
# model.add(Conv2D(512,(3,3), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

model.add(Flatten())

# Fully connected Block 1
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected Block 2
# model.add(Dense(512))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

# opt = Adam(lr=0.0001)
# opt = Adam(lr= 0.00010000000474974513)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [55]:
opt = keras.optimizers.SGD(learning_rate=0.005)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
history = model.fit(images , labels , batch_size=64 , epochs=200 ,class_weight = class_weights, validation_data = (t_images ,t_labels))

In [56]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [40]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()